In [0]:
df_users_bronze_history = spark.sql("DESCRIBE HISTORY slack_etl.bronze.users LIMIT 1")
users_bronze_latest_commit_version = df_users_bronze_history.orderBy('version', ascending=False).limit(1).collect()[0].asDict()['version']

In [0]:
df_users_bronze = ( 
                   spark.read.format('delta')
                    # .option('readChangeData','true')
                    #.option('startingVersion', users_bronze_latest_commit_version)
                    .table('slack_etl.bronze.users')
)


In [0]:

df_users_silver_enriched = df_users_bronze.selectExpr("created","id","profile.display_name","profile.real_name","profile.email","profile.fields.department.value as department","profile.fields.location.value as location", "team_id")

In [0]:
df_users_silver_enriched.write.mode('overwrite').saveAsTable('slack_etl.silver.users')

In [0]:
%sql
SELECT * FROM slack_etl.silver.users